In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL-Advanced


# 튜토리얼

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 {ability} 에 능숙한 어시스턴트입니다. 20자 이내로 응답하세요",
        ),
        # 대화 기록을 변수로 사용, history 가 MessageHistory 의 key 가 됨
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # 사용자 입력을 변수로 사용
    ]
)
runnable = prompt | model  # 프롬프트와 모델을 연결하여 runnable 객체 생성

# 휘발성 대화기록: 인메모리(In-Memory)

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}  # 세션 기록을 저장할 딕셔너리

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store: # 세션 ID가 store에 없는 경우
        store[session_ids] = ChatMessageHistory()
    return store[session_ids] # 해단 세션 ID에 대한 세션 기록 반환

with_message_history = (
    RunnableWithMessageHistory( # RunnableWithMessageHistory 객체 생성
        runnable,
        get_session_history, # 세션 기록을 가져오는 함수
        input_messages_key="input", # 입력 메시지의 키
        history_messages_key="history", # 기록 메시지의 키
    )
)

In [7]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)

abc123


AIMessage(content='It is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 47, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMA29Rs2J2ZXYoA0zLBOWme3D7Qdi', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ba6e17b-1225-4a23-9518-3ed7c01ff669-0', usage_metadata={'input_tokens': 47, 'output_tokens': 26, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
# 메시지 기록을 포함하여 호출합니다.
with_message_history.invoke(
    # 능력과 입력을 설정합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요."},
    # 설정 옵션을 지정합니다.
    config={"configurable": {"session_id": "abc123"}},
)

abc123


AIMessage(content='인접변을 빗변으로 나눈 비율을 나타내는 삼각함수입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 97, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMA2Wu88k7gVgVebugfRYHXae5GTE', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e78509d-e55a-44ca-a6cc-9d1432e5c97a-0', usage_metadata={'input_tokens': 97, 'output_tokens': 35, 'total_tokens': 132, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
# 새로운 session_id로 인해 이전 대화 내용을 기억하지 않습니다.
with_message_history.invoke(
    # 수학 능력과 입력 메시지를 전달합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요"},
    # 새로운 session_id를 설정합니다.
    config={"configurable": {"session_id": "def234"}},
)

def234


AIMessage(content='수학에 능숙한 어시스턴트입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 58, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMA3UbHaBWlbfIo77uAuWZvSc5u9P', 'finish_reason': 'stop', 'logprobs': None}, id='run-640cd983-2a77-4855-bb31-422a7201a231-0', usage_metadata={'input_tokens': 58, 'output_tokens': 20, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}  # 빈 딕셔너리를 초기화합니다.


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    # 주어진 user_id와 conversation_id에 해당하는 세션 기록을 반환합니다.
    if (user_id, conversation_id) not in store:
        # 해당 키가 store에 없으면 새로운 ChatMessageHistory를 생성하여 저장합니다.
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id", # get_session_history 함수의 첫 번째 인자로 사용
            annotation=str,
            name="User ID",
            description="사용자의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            default="",
            description="대화의 고유 식별자입니다.",
            is_shared=True,
        )
    ]
)

In [11]:
with_message_history.invoke(
    # 능력(ability)과 입력(input)을 포함한 딕셔너리를 전달합니다.
    {"ability": "math", "input": "Hello"},
    # 설정(config) 딕셔너리를 전달합니다.
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

AIMessage(content='안녕하세요. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 43, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMA6WWnbbpw5niRqzlobKnDKwEyk8', 'finish_reason': 'stop', 'logprobs': None}, id='run-356bbb6c-12ca-40a9-a19c-a90fa333de6b-0', usage_metadata={'input_tokens': 43, 'output_tokens': 22, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# 다양한 Key를 사용한 Runnable 을 사용한 예시

In [13]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

# chain 생성
chain = RunnableParallel({"output_message": ChatOpenAI()})

In [14]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # 세션 ID에 해당하는 대화 기록이 저장소에 없으면 새로운 ChatMessageHistory를 생성합니다.
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    # 세션 ID에 해당하는 대화 기록을 반환합니다.
    return store[session_id]

In [18]:
# 체인에 대화 기록 기능을 추가한 RunnableWithMessageHistory 객체를 생성합니다.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    output_messages_key="output_message",
)

# 주어진 메시지와 설정으로 체인을 실행합니다.
with_message_history.invoke(
    # 혹은 "what is the definition of cosine?" 도 가능
    [HumanMessage(content="what is the definition of cosine?")],
    config={"configurable": {"session_id": "abc123"}},
)

{'output_message': AIMessage(content='The cosine of an angle in a right triangle is the ratio of the length of the side adjacent to the angle to the length of the hypotenuse. It is a trigonometric function that relates the angle measurement to the sides of a right triangle. The cosine function is denoted as cos and is widely used in mathematics and physics for solving problems involving angles and distances.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 344, 'total_tokens': 420, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMA8qkfuxPoQvjPJKxLLrbLabBPG5', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c332123-fa9d-423c-aeca-ebe60d7458c1-0', usage_metadata={'input_tokens': 344

In [16]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="이전의 내용을 한글로 답변해 주세요!")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)

{'output_message': AIMessage(content='삼각법에서의 코사인은 직각삼각형에서의 각의 측면 중에서 인접한 측면의 길이를 빗변의 길이로 나눈 것으로 정의됩니다. 코사인은 각의 측정을 직각 삼각형의 측면과 관련시키는 기본적인 삼각함수이며, 코사인 함수는 각도와 거리에 관련된 문제를 해결하기 위해 수학과 물리학에서 널리 사용됩니다. 코사인 함수는 cos로 표시되며, 각과 거리를 다루는 문제를 푸는 데에 자주 사용됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 208, 'prompt_tokens': 122, 'total_tokens': 330, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMA8MkFQcXA6vB2B4l0PmPHiKIuvt', 'finish_reason': 'stop', 'logprobs': None}, id='run-c66dc6d9-c205-442b-a656-d80d4e9da203-0', usage_metadata={'input_tokens': 122, 'output_tokens': 208, 'total_tokens': 330, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}

In [19]:
with_message_history = RunnableWithMessageHistory(
    ChatOpenAI(),  # ChatOpenAI 언어 모델을 사용합니다.
    get_session_history,  # 대화 세션 기록을 가져오는 함수를 지정합니다.
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    # output_messages_key="output_message",
)

In [20]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="코사인의 의미는 무엇인가요?")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "def123"}},
)

AIMessage(content='코사인은 삼각함수 중 하나로, 직각삼각형의 두 변의 길이를 이용하여 각을 구하는 방법 중 하나입니다. 코사인은 "인접 변의 길이 / 빗변의 길이"로 계산되며, 각의 크기에 따라 -1에서 1 사이의 값을 가집니다. 코사인 함수는 주로 삼각형의 각을 구할 때 사용되지만, 물리학이나 공학 분야에서도 다양하게 활용됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 24, 'total_tokens': 188, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMA9WoccPMpuT2vMYet88WnWuxIi4', 'finish_reason': 'stop', 'logprobs': None}, id='run-3d32e344-2821-48f4-b8a8-0108d76c0d91-0', usage_metadata={'input_tokens': 24, 'output_tokens': 164, 'total_tokens': 188, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [21]:
from operator import itemgetter

with_message_history = RunnableWithMessageHistory(
    # "input_messages" 키를 사용하여 입력 메시지를 가져와 ChatOpenAI()에 전달합니다.
    itemgetter("input_messages") | ChatOpenAI(),
    get_session_history,  # 세션 기록을 가져오는 함수입니다.
    input_messages_key="input_messages",  # 입력 메시지의 키를 지정합니다.
)

In [22]:
with_message_history.invoke(
    {"input_messages": "코사인의 의미는 무엇인가요?"},
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "xyz123"}},
)

AIMessage(content='코사인은 삼각함수 중 하나로, 직각삼각형에서의 각도와 그 각도에 해당하는 변의 길이 사이의 비를 나타내는 함수입니다. 각도가 변화할 때 코사인 값도 변화하며, 0도에선 1, 90도에선 0, 180도에선 -1이 됩니다. 이러한 성질로 코사인은 여러 분야에서 활용되며, 특히 삼각함수와 함께 삼각법 문제를 해결하는 데 사용됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 172, 'prompt_tokens': 24, 'total_tokens': 196, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMAA2C8OGOneZMbZIdjQK5LYs10S7', 'finish_reason': 'stop', 'logprobs': None}, id='run-7bb64371-6dd7-4cf8-88c7-9eb7f5494b02-0', usage_metadata={'input_tokens': 24, 'output_tokens': 172, 'total_tokens': 196, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})